# Emulation of nonlinear audio systems with LSTM in Real Time
version : LSTM9

Imports :

In [1]:
import sys,select,os
sys.path.append('./Codes')
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import scipy.io.wavfile
import time
import pyaudio 
from tensorflow.python.client import timeline
from tensorflow.contrib.rnn import *



#pathModel = './Experiments/2018-11-02-14-21/temp/' #(200,150)
#pathModel = './Experiments/2019-01-06-13-03/temp/' #(200,150) reduced 8
#pathModel = './Experiments/2019-01-06-16-15/temp/' #(100,100) reduced 8
#pathModel = './Experiments/2019-04-26-13-34/temp/' #(Nh=100,N=100) model 1
#pathModel = './Experiments/2019-04-29-11-28/temp/' #(Nh=100,N=100) model 2
#pathModel = './Experiments/2019-05-10-14-26/temp/' #(Nh=100,N=50) model 1 (test speed 1 epoch)
#pathModel = './Experiments/2019-05-10-14-29/temp/' #(Nh=100,N=20) model 1 (test speed 1 epoch)
#pathModel = './Experiments/2019-05-10-14-34/temp/' #(Nh=100,N=10) model 1 (test speed 1 epoch)
#pathModel = './Experiments/2019-05-17-12-34/temp/' #(Nh=150,N=150) model 8 (test 1 epoch)
pathModel = './Experiments/2019-05-17-14-03_model8/temp/' #(Nh=150,N=150) model 8  keras





if not os.path.isdir(pathModel):
    print("could nt find path model {}".format(pathModel))
num_step = 150
bufferAudioSize = 400
fs = 22050


with tf.Session() as sess:
    #config=tf.ConfigProto(log_device_placement=True)
    saver = tf.train.import_meta_graph(pathModel+'myModel.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint(pathModel))
    lastBuff = np.zeros((num_step-1,))
    graph = tf.get_default_graph()
    data = graph.get_tensor_by_name("placeHolder/data:0")
    #is_training = graph.get_tensor_by_name("placeHolder/is_training:0")
    #prediction = graph.get_tensor_by_name("FCLayer/prediction:0")
    prediction = graph.get_tensor_by_name("FCPred/Tanh:0")
    #[print(n.name) for n in tf.get_default_graph().as_graph_def().node]
    
    #options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    #run_metadata = tf.RunMetadata()
    
    dataNonShaped = tf.placeholder(tf.float32, [bufferAudioSize,],name="inputFromADC")
    
    prevBuff = tf.placeholder(tf.float32, [num_step-1,],name = "EndofPreviousInputBufferNonShaped")
    #with tf.device('/cpu:0'):
    newBuff = tf.concat([prevBuff,dataNonShaped],0) #size num_step-1+bufferAudioSize
    nextBuff = newBuff[-(num_step-1):] # save for next iteration
    
    my_indices = tf.constant(np.arange(bufferAudioSize))
    indices = (np.arange(num_step) +my_indices[:,tf.newaxis])
    dataShaped = tf.gather(newBuff,indices) # slice input vector into tensor of shape(audiobufferSize,num_step)
    #print(dataShaped.get_shape())
    r=0
    timeTot=0
    def callback(in_data, frame_count, time_info, flag):
        if flag:
            print("Playback Error: {}".format(flag))
        global lastBuff
        global r
        global timeTot
        tstart = time.time()
        audio_data = np.frombuffer(in_data, dtype=np.float32)
        
        dataShapedToProcess,lastBuff = sess.run([dataShaped,nextBuff], feed_dict={dataNonShaped : audio_data, prevBuff : lastBuff})

        
        #out = sess.run(prediction, feed_dict={data: dataShapedToProcess}, options=options, run_metadata=run_metadata)
        out = sess.run(prediction, feed_dict={data: dataShapedToProcess})
        tstop = time.time()
        timeIt = tstop-tstart
        timeTot+=timeIt
        r+=1
        if(r % 100==0):
            print("out:{}".format(timeTot/r))
            r=0
            timeTot=0
        
            # Create the Timeline object, and write it to a json file
        #fetched_timeline = timeline.Timeline(run_metadata.step_stats)
        #chrome_trace = fetched_timeline.generate_chrome_trace_format()
        #with open('timeline_01.json', 'w') as f:
        #    f.write(chrome_trace)
        return out, pyaudio.paContinue
    
    #fake graph use before launch audio stream, to place node on the graph
    for x in range(0, 5):
        initInput = [0]*bufferAudioSize
        dataShapedToProcess = sess.run([dataShaped], feed_dict={dataNonShaped : initInput, prevBuff : lastBuff})
        _ = sess.run(prediction, feed_dict={data: dataShapedToProcess[0]})
    
    pa = pyaudio.PyAudio()
    
    stream = pa.open(format = pyaudio.paFloat32,
                     channels = 1,
                     rate = fs,
                     output = True,
                     input = True,
                     frames_per_buffer = bufferAudioSize,
                     stream_callback = callback)
    print("input latency {} s".format(stream.get_input_latency()))
    print("output latency {} s".format(stream.get_output_latency()))

    #stream.start_stream()
    while stream.is_active():   
        time.sleep(0.1)

    print("stream interrupted")   
    stream.stop_stream()
    stream.close()
    pa.terminate()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./Experiments/2019-05-17-14-03_model8/temp/myFinalModel.ckpt
input latency 0.036281179138321996 s
output latency 0.054421768707483 s
out:0.003820669651031494
out:0.00417816162109375
out:0.003914952278137207
out:0.003963921070098877
out:0.004230132102966308
out:0.0037457418441772462
out:0.00412644624710083
out:0.0040575599670410155
out:0.003852982521057129
out:0.0041788649559021
out:0.003989768028259277
out:0.003940165042877197
out:0.003917124271392822
out:0.003921763896942139


RuntimeError: Attempted to use a closed Session.

RuntimeError: 